In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split
import pandas as pd

spark = (SparkSession.builder
         .appName("uber-date-trips")
         .master("local[2]")
         .getOrCreate()
         )

24/03/25 16:31:32 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
trip_data_2020_03_path = '../../data/fhvhv_tripdata_2020-03.parquet'

# auth check schema & mapping header
df = spark.read.options(inferSchema=True, header=True).load(trip_data_2020_03_path)

In [3]:
df.show(5)

+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+--

In [4]:
df.createOrReplaceTempView("mobility_data")

In [6]:
spark.sql("""
    SELECT * from mobility_data LIMIT 5
""").show()

+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+--

In [13]:
# 일별 Trip 수
spark.sql("""
    SELECT split(pickup_datetime, ' ')[0] as pickup_date,
           count(1)
    FROM mobility_data
    GROUP BY pickup_date
    ORDER BY pickup_date
""").show(5)

+-----------+--------+
|pickup_date|count(1)|
+-----------+--------+
| 2020-03-01|  784260|
| 2020-03-02|  648990|
| 2020-03-03|  697880|
| 2020-03-04|  707879|
| 2020-03-05|  731165|
+-----------+--------+


In [16]:
pd.read_csv("../../data/taxi_zone_lookup.csv").head(5)

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [17]:
zone_lookup_df = spark.read.options(inferSchema=True, header=True).format('csv').load("../../data/taxi_zone_lookup.csv")

In [18]:
zone_lookup_df.createOrReplaceTempView('zone_data')

In [19]:
spark.sql("""
    SELECT *
    FROM zone_data LIMIT 5
""").show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+


In [23]:
# Location 별 승차 수

spark.sql("""
    SELECT
        t2.Borough,
        count(1)
    FROM mobility_data as t1
    INNER JOIN zone_data as t2 ON t1.PULocationID = t2.LocationID
    GROUP BY t2.Borough
    ORDER BY 2 desc
""").show()


+-------------+--------+
|      Borough|count(1)|
+-------------+--------+
|    Manhattan| 4953147|
|     Brooklyn| 3735765|
|       Queens| 2437394|
|        Bronx| 2086597|
|Staten Island|  178818|
|          N/A|     845|
|          EWR|     362|
+-------------+--------+


In [25]:
# Location 별 하차 수

spark.sql("""
    SELECT
        t2.Borough,
        count(1)
    FROM mobility_data as t1
    INNER JOIN zone_data as t2 ON t1.DOLocationID = t2.LocationID
    GROUP BY t2.Borough
    ORDER BY 2 desc
""").show()

+-------------+--------+
|      Borough|count(1)|
+-------------+--------+
|    Manhattan| 4553783|
|     Brooklyn| 3696684|
|       Queens| 2468416|
|        Bronx| 2043492|
|          N/A|  387759|
|Staten Island|  177727|
|          EWR|   65066|
|      Unknown|       1|
+-------------+--------+
